# Process texts

В цьому ноутбуці ми завантажимо,  перекладемо, почистимо тексти

In [39]:
# Preprocess texts

### Clean texts

import re

import demoji
import pandas as pd
import numpy as np
import re
import string
import cld2
from numpy import isin
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from lemmatization import process, process_pipeline


# demoji.download_codes()

whitespace = re.compile("[\s\u0020\u00a0\u1680\u180e\u202f\u205f\u3000\u2000-\u200a]+", re.UNICODE)

def simple_tokenize(text, max_shrink=False):
    """
    Tokenizer for sentiment analysis of mentions (MOSTLY Facebook text)
    """
#     delete urls
    clean_text = re.sub(r'http\S+', ' ', text)
#     delete everything in brackets -- <>
    clean_text = re.sub(r"</?\w*>", " ", clean_text)
#     delete
    for c in ["(", ")", "!", "?", r"-", r"*", ".", ",", r'"']:
        clean_text = re.sub("[%s]+" % c, c, clean_text)

#     delete hashtags
    clean_text = re.sub(r"#\w*|№\w*", " ", clean_text)
    
#     delete complex model names
    clean_text = re.sub(r"\b\S+[0-9]+\S+\b", " ", clean_text)

    if clean_text == '':
        return "NULL"

    clean_text = clean_text.replace("\n", ' ')
    clean_text = clean_text.replace("\b", " ")
    clean_text = clean_text.replace("=", " ")
    clean_text = clean_text.replace("+", " ")

    # add spaces around punctuation
    chars = string.punctuation
    chars = chars.replace("`", "")
    chars = chars.replace("'", "")
    chars += "«»\"“”‘’.?!…,:;"

    for c in chars:
        clean_text = clean_text.replace(c, ' ')

#     delete emojis
    clean_text = demoji.replace(clean_text)
#     remove digits
    clean_text = ''.join(ch for ch in clean_text if ch not in string.digits)
#     set to lowercase
    clean_text = clean_text.lower()

    if clean_text.replace(' ', '') == '':
        return "NULL"

    only_letters = re.sub(f"[0-9]|[{chars}]|\s", "", clean_text)

    if only_letters == '' or len(only_letters) == 0:
        return "NULL"
    else:
        if max_shrink:
            return re.sub(f"[0-9]|[{chars}]", "", clean_text)
        else:
            return clean_text


def squeeze_whitespace(input_):
    return whitespace.sub(" ", input_).strip()


def tokenize(text, max_shrink=False):
    """
    Function refers to splitting up a larger body of text into words
    text: parameter takes sentence
    max_shrink: leaves only letters
    """
    try:
        assert isinstance(text, str)
    except AssertionError:
        return None
    else:
        return squeeze_whitespace(simple_tokenize(text, max_shrink))


def predict_languages_with_cld2(text):
    """
    # BROKEN LOGIC
    text: str
    """
    assert isinstance(text, str)

    try:
        output = cld2.detect(text)
    except ValueError:
        return "Unknown"

    pred_lang = output.details[0].language_name
    if pred_lang == "RUSSIAN":
        return "ru"
    elif pred_lang == "UKRAINIAN":
        return "uk"
    elif pred_lang == "ENGLISH":
        return "en"
    else:
        return "Unknown"


def main(df):
#     cleaning
    df["clean_text"] = df.text.apply(lambda x: tokenize(x))

#     remove empty texts
    print(f"Total number of empty texts is -- {df[df.clean_text == 'NULL'].shape[0]}")
    df = df[df.clean_text != "NULL"]
    
#     frop NAs from the dataframe
    df.dropna(subset=["clean_text"], inplace=True)

#     Predict languages
    df['lang'] = df.clean_text.apply(lambda x: predict_languages_with_cld2(x))

    df = df[isin(df.lang.tolist(), ["ru", "uk"])]

    ### Translate
    # red translated texts
    df_rus = pd.read_excel(r"C:\Users\BHaiov01\Documents\Projects\Analysis\gorenje_2022\translations\november\gorenje_ready_for_translation.xlsx", 
                           index_col=0)
    print(f"Loaded {df_rus.shape[0]} translated rows")

#     leave only indexes that are needed
    df_rus = df_rus[isin(df_rus.index, df.index)]
    df = df[isin(df.index, df_rus.index)]

#     clean translated texts
    df_rus["clean_rus"] = df_rus.translated_text.apply(lambda x: tokenize(x))

    ### Remove stopwords

    with open("helpdata/stop_words_russian.txt", "r") as f:
        stop_words = [w.replace("\n", "") for w in f.readlines()]

    c = 0
    for w in stopwords.words("russian"):
        if w in stop_words:
            c += 1
        else:
            stop_words.append(w)
    print("Total number of stopwords is", len(stop_words))

    df_rus["clean_rus_no_stopwords"] = df_rus.clean_rus.apply(lambda x: " ".join([w for w in x.split() if w not in stop_words]))
    
    # lets remove russian names

    russian_names = pd.read_csv("russian_names.csv", sep=';')
    russian_names.drop(columns=['ID', 'Sex', 'PeoplesCount', 'WhenPeoplesCount', 'Source'], inplace=True)
    russian_names = russian_names.Name.tolist()
    print("Loaded", len(russian_names), "unique russian names")

    df_rus["clean_rus_no_stopwords"] = df_rus.clean_rus_no_stopwords.apply(lambda x: " ".join([w for w in x.split() if w not in russian_names]))
    
    df_rus["index"] = df_rus.index

    ### Lemmatization
    lemmatized = {}

    # {text_index : [list of lemmatized words], ...}
    for index, text in zip(df_rus.index, df_rus.clean_rus_no_stopwords.tolist()):
        lemmatized_words = process(process_pipeline, text, keep_pos=True)
        lemmatized[index] = [w for w in lemmatized_words if w.split("_")[0] not in stop_words and \
                             w.split("_")[0] not in russian_names]
        
    df_rus["lemmatized"] = None

    for index in df_rus.index:
        df_rus.loc[index, "lemmatized"] = " ".join(lemmatized[index])
        
    return df, df_rus

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BHaiov01\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##### Завантажимо дані та викинемо дублікати

In [3]:
df = pd.read_excel("november_raw_data.xlsx", index_col=0)

# df.head()
print(f"Loaded {df.shape[0]} rows")

df.drop(columns=['Страна',
       'Регион', 'Город', 'Уровень издания', 'Рубрика', 'Язык',
       'Дата постановки', 'Ссылка', 'Время начала (ТВ)',
       'Время окончания (ТВ)', 'Аудитория/тираж', 'Аннотация', 'Дубли', 'Код',
       'Cредняя посещаемость', 'Аудитория автора', 'Аудитория сообщества'], inplace=True)
df.columns = ["title", "text", "source_type", "source", "date"]

# drop duplicated rows
df.drop_duplicates(subset=["text"], keep="first", inplace=True)
print(f"Number of unique rows is -- {df.shape[0]}")

Loaded 32071 rows
Number of unique rows is -- 22263


##### ф-ція для порівняння тексту та регулярного виразу

In [4]:
def contains_mention(text):
    """
    Function checks if there is a mention of tags in the text
    """
    if re.findall(req.lower(), text.lower()) == []:
        return False
    else:
        return True

contains_mention_MP = np.vectorize(contains_mention)

##### Оберемо тільки ті рядочки, які відповідають запиту для побутової техніки

In [5]:
from requests_finder import load_request

req = load_request(fname="request.txt")
df["home_appliances"] = contains_mention_MP(df.text.values)
print("Number of texts related to home appliances --", df[df.home_appliances == 1].shape[0])
df = df[df.home_appliances == 1]

Number of texts related to home appliances -- 7311


##### Проставимо 1-й набір тегів

In [6]:
all_themes = pd.read_excel("tags_gorenje.xlsx", sheet_name="MAIN_1")

for index, row in all_themes.iterrows():
    req = load_request(request=row["Примеры ключевых слов"])
    theme = row["Категории"]
    print(f"{index + 1}/{all_themes.shape[0]} | {theme}")
    df[f"MAIN_1: {theme}"] = contains_mention_MP(df.text.values)

1/13 | НЕРЕЛЕВАНТ
2/13 | Продаж нової техніки
3/13 | Продаж Б/У
4/13 | Ремонт/Поломки/Деталі
5/13 | Послуги
6/13 | Огляд
7/13 | Розіграші
8/13 | Продажа(здача) квартир з подутовою технікою
9/13 | Корп активність в компанії
10/13 | Софти
11/13 | Відгуки в інтернет-магазинах (розширити тематику)
12/13 | Вакансії/робота
13/13 | інші електронні пристрої


##### Проставимо 2-й набір тегів

In [7]:
all_themes_2 = pd.read_excel("tags_gorenje.xlsx", sheet_name="MAIN_2")

for index, row in all_themes_2.iterrows():
    req = load_request(request=row["Примеры ключевых слов"])
    theme = row["Категории"]
    print(f"{index + 1}/{all_themes_2.shape[0]} | {theme}")
    df[f"MAIN_2: {theme}"] = contains_mention_MP(df.text.values)

1/12 | Холодильник
2/12 | Пральна машина
3/12 | Сушка
4/12 | Посудомийка
5/12 | Морозильна камера
6/12 | Духовка
7/12 | Варочні поверхні
8/12 | Витяжка
9/12 | Мікрохвильовка
10/12 | Кондиціонер
11/12 | Пилосос
12/12 | Бойлер


In [48]:
load_request(request=all_themes_2.loc[1, "Примеры ключевых слов"])

'(\\bстирал\\w*.{1,5}\\bмашин\\w*|\\bмашин\\w*.{1,5}\\bстирал\\w*|\\bпральн\\w*.{1,5}\\bмашин\\w*|\\bмашин\\w*.{1,5}\\bпральн\\w*|\\bпранн\\w*.{1,5}\\bмашин\\w*|\\bмашин\\w*.{1,5}\\bпранн\\w*|\\bстирк\\w*.{1,5}\\bмашин\\w*|\\bмашин\\w*.{1,5}\\bстирк\\w*|\\bстиралк\\w*|\\bпрал\\w*)'

In [49]:
all_themes_2.loc[1, "Примеры ключевых слов"]

'((стирал* OR пральн* OR пранн* OR стирк*) /5 машин*) OR стиралк*  OR прал*'

##### Обробимо тексти

In [40]:
df_processed, df_rus = main(df)

Total number of empty texts is -- 29


<ipython-input-39-7f800740aab5>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=["clean_text"], inplace=True)
<ipython-input-39-7f800740aab5>:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lang'] = df.clean_text.apply(lambda x: predict_languages_with_cld2(x))


Loaded 29815 translated rows
Total number of stopwords is 572
Loaded 51529 unique russian names


##### Додаткова перевірка обробки

Перевіряються короткі слова. Якщо слово менше або рівне 3-х символів та не є словом із головного запиту, ми його видаляємо 

In [41]:
from string import digits
from requests_finder import load_request
import re

def decompose_initial_request(path_to_req):
    with open(path_to_req, "r", encoding="utf8") as f:
              req = f.readlines()[0]
    
    new_req_list = []
    req = req.replace("(", "")
    req = req.replace(")", "")
    req = req.replace("/", "")
    req = req.replace("AND", "")
    req = req.replace("OR", "")
    for digit in digits:
        req = req.replace(str(digit), "")
    
    for t in req.split():
        new_req_list.append(t)
        
    new_req = " OR ".join(new_req_list)
    
    return new_req

In [42]:
WORD_MIN_LENGTH = 2

lemmatized = {}
for index in df_rus.index:
    lemmatized[index] = df_rus.loc[index, "lemmatized"].split(" ")

vocab = []
for tokens in lemmatized.values():
    for word in tokens:
        if word not in vocab:
            vocab.append(word)

print(f"Total length of vocab is {len(vocab)}")

req = decompose_initial_request(path_to_req="request.txt")
req = load_request(request=req)

popped_words = []
for word in vocab:
    postag = word.split("_")[1]
    word = word.split("_")[0]
    if len(word) <= WORD_MIN_LENGTH:
        if not contains_mention(word):
            popped_words.append(f"{word}_{postag}")
            vocab.remove(f"{word}_{postag}")
print(f"{len(popped_words)} words were popped")

for index, text in lemmatized.items():
    text = [w for w in text if w not in popped_words]
    lemmatized[index] = text
    df_rus.loc[index, "lemmatized"] = " ".join(text)

print("Data edited and saved")

Total length of vocab is 28241
692 words were popped
Data edited and saved


In [44]:
df_rus["lemmatized_with_pos"] = df_rus["lemmatized"].copy()
df_rus["lemmatized"] = df_rus["lemmatized"].apply(lambda x: " ".join([w.split("_")[0] for w in x.split(" ")]))

In [46]:
df_rus.head(2)

,mention,clean_text,language,clipped_part_correct,translated_text,clean_rus,clean_rus_no_stopwords,index,lemmatized,lemmatized_with_pos
0,Инверторные кондиционеры | Купить в Москве по ...,инверторные кондиционеры | купить в москве по ...,ru,NaN,инверторные кондиционеры | купить в москве по ...,инверторные кондиционеры купить в москве по вы...,инверторные кондиционеры купить москве выгодно...,0,инверторный кондиционер покупать москве выгодн...,инверторный_ADJ кондиционер_NOUN покупать_VERB...
1,Пральна машина lg f1296hds1 12 199 грн. https:...,пральна машина lg fhds грн . пральна машина lg...,uk,NaN,Стиральная машина LG FHDS HR. Стиральная машин...,стиральная машина lg fhds hr стиральная машина...,стиральная машина lg fhds hr стиральная машина...,1,стиральный машина lg fhds стиральный машина lg...,стиральный_ADJ машина_NOUN lg_X fhds_X стираль...


##### Збережемо

In [47]:
writer = pd.ExcelWriter("home_appliances_with_tags.xlsx",
                        engine='xlsxwriter',
                        options={'strings_to_urls': False, 'strings_to_formulas': False})
df_processed.to_excel(writer)
writer.close()

writer = pd.ExcelWriter("home_appliances_rus.xlsx",
                        engine='xlsxwriter',
                        options={'strings_to_urls': False, 'strings_to_formulas': False})
df_rus.to_excel(writer)
writer.close()